In [1]:
from dataset import torch, os, LocalDataset, transforms, np, get_class, num_classes, preprocessing, Image, m, s
from config import *

from torch import nn
from torch.optim import SGD
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.models import resnet, vgg

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from matplotlib import pyplot as plt
from numpy import unravel_index
import gc
import argparse


In [2]:
mean=m
std_dev=s

transform = transforms.Compose([transforms.Resize((224,224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean, std_dev)])

In [3]:
classes = {"num_classes": len(num_classes)}
resnet152_model = resnet.resnet152(pretrained=False, **classes)
model_name="resnet152"
model=resnet152_model

print (str(RESULTS_PATH) + "/" + str(model_name) + "/" + str(model_name) + ".pt")
model.load_state_dict(torch.load(str(RESULTS_PATH) + "/" + str(model_name) + "/" + str(model_name) + ".pt"))

results/resnet152/resnet152.pt


In [4]:
def test_sample(image_path, model=model, model_name=model_name):
    im = Image.open(image_path).convert("RGB")
    im = transform(im)

    if USE_CUDA and cuda_available:
        model = model.cuda()
    model.eval()

    x = Variable(im.unsqueeze(0))

    if USE_CUDA and cuda_available:
        x = x.cuda()
        pred = model(x).data.cuda().cpu().numpy().copy()
    else:
        pred = model(x).data.numpy().copy()

    #print (pred)

    idx_max_pred = np.argmax(pred)
    idx_classes = idx_max_pred % classes["num_classes"]
    #print(get_class(idx_classes))
    return get_class(idx_classes)

In [5]:
test_sample("sample/fordx.jpg")

'1_ford_explorer'

In [6]:
test_sample("sample/ford.jpg")

'1_ford_explorer'

In [7]:
test_sample("sample/honda.jpg")

'3_honda_civic'

In [8]:
test_sample("sample/hondax.jpg")

'3_honda_civic'

In [9]:
test_sample("sample/honday.jpg")

'3_honda_civic'

In [10]:
test_sample("sample/nissan.jpg")

'2_nissan_altima'

In [11]:
test_sample("sample/nissan2.jpg")

'2_nissan_altima'

In [12]:
test_sample("sample/nissan_1.jpg")

'2_nissan_altima'

In [13]:
test_sample("sample/nissanx.jpg")

'2_nissan_altima'

## Test con immagini prese dal web

In [14]:
test_sample("sample/ford_web.png")

'1_ford_explorer'

In [15]:
test_sample("sample/honda_web.jpeg")

'3_honda_civic'

In [16]:
test_sample("sample/nissan_web.jpeg")

'2_nissan_altima'

In [17]:
test_sample("sample/ford_explor.jpg")

'1_ford_explorer'

In [18]:
from glob import glob
ford = glob("test_images/Ford_Explorer/*")
nissan = glob("test_images/Nissan_Altima/*")
honda = glob("test_images/Honda_Civic/*")

In [19]:
ford_results = ""
for f in ford:
    ford_results += f+","+test_sample(f)+"\n"

In [20]:
nissan_results = ""
count = 0
for f in nissan:
    out = test_sample(f)
    if out == "2_nissan_altima":
        count+=1
    nissan_results += f+","+out+"\n"
print (count)

62


In [21]:
honda_results = ""
for f in honda:
    honda_results += f+","+test_sample(f)+"\n"

In [22]:
file = open("ford.csv", "w+")
file.write(ford_results)
file.close()

In [23]:
file = open("nissan.csv", "w+")
file.write(nissan_results)
file.close()

In [24]:
file = open("honda.csv", "w+")
file.write(honda_results)
file.close()